In [1]:
import os 
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import LeakyReLU
import tensorflow_datasets as tdfs
import matplotlib.pyplot as plt
from rnnconv import RnnConv

print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images2, train_labels2), (test_images2, test_labels2) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
train_images2, test_iamges2 = train_images2 / 255.0, test_images2 / 255.0


train_images = np.concatenate((train_images, train_images2) ,axis=0)
test_images = np.concatenate((test_images, test_images2), axis=0)

train_labels = np.concatenate((train_labels, train_labels2), axis=0)# Data Augmentation

rot90 = np.rot90(train_images,axes=(1,2))
rot180 = np.rot90(train_images,k=2,axes=(1,2))
rot270 = np.rot90(train_images,k=3,axes=(1,2))

train_images_pro = np.concatenate((train_images, rot90) ,axis=0)
train_images_pro.shape

(200000, 32, 32, 3)

In [3]:
def residual_loss(y_true, y_pred):
    B = batch_size
    s = B * H * W * C
    n = iterations
    beta = 1 / (s * n)
    
    summation = tf.math.reduce_sum(tf.math.abs(tf.math.subtract(y_true, y_pred)), [0,1,2,3])
    return beta * summation

def SSIMLoss(y_true, y_pred):
    return tf.math.square(1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0)))

def PSNRLoss(y_true, y_pred,max_val=1.0):
    return tf.image.psnr(y_true, y_pred, max_val=max_val)

def SSIM_res_loss(y_true, y_pred):
    return SSIMLoss(y_true, y_pred) + residual_loss(y_true, y_pred)

def test_model(model, test_images, indexes=np.arange(5)):
    img_preds = model.predict(test_images[indexes])
    
    plt.figure(figsize=(10,10))
    for i in indexes:
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(test_images[i], cmap=plt.cm.binary)
    plt.show()

    plt.figure(figsize=(10,10))
    for i in indexes:
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img_preds[i], cmap=plt.cm.binary)
    plt.show()
    
def plot_metric(model, metric='val_SSIMLoss'):
    import matplotlib.pyplot as plt
    plt.plot(model.history[metric])

def save_model(model, path):
    model.save(path)
    print(f'Model saved to: {path}.')
    
def load_model(path, custom_objects):
    '''
    path is the directory containing a "saved_model.pb" - E.g. path=models/new_model"
    Custom objects must be any user-defined loss functions. E.g. {'SSIMLoss': SSIMLoss, 'PSNRLoss': PSNRLoss, 'residual_loss': residual_loss}
    '''
    return tf.keras.models.load_model(path, custom_objects=custom_objects)

def save_history(history, path):

    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to csv: 
    hist_csv_file = f'{path}/history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

In [4]:
def get_model(iterations):
    
    model = tf.keras.Sequential() 

    for i in range(iterations):

        model.add(layers.Conv2D(filters=64, kernel_size=3, input_shape=(32, 32, 3), strides = (2, 2), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(2, 2),   padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Conv2D(filters=32, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Conv2D(filters=512, kernel_size=1,  strides = (1,1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=512, kernel_size=2, strides=(1, 1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))
        model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=512, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))
        model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))


        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=256, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))
        model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

        model.add(layers.Lambda(lambda x: tf.expand_dims(x,axis=1)))
        model.add(layers.ConvLSTM2D(filters=128, kernel_size=3, strides=(1, 1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.01))
        model.add(layers.Lambda(lambda x:tf.nn.depth_to_space(x,2,data_format='NHWC')))

        model.add(layers.Conv2D(filters=3, kernel_size=1, strides = (1,1), padding='same', data_format='channels_last'))
        model.add(LeakyReLU(alpha=0.1))
        
    opt = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate, 
                                   beta_1=0.9, beta_2=0.999, 
                                   epsilon=1e-06, amsgrad=True)
    
    model.compile(optimizer=opt, 
                  loss=SSIM_res_loss, 
                  metrics=[SSIMLoss, PSNRLoss, residual_loss, "mae", "acc"])
        
    return model

In [5]:
epochs=5
iterations = 1
batch_size = 512
H = train_images.shape[1]
W = train_images.shape[2]
C = train_images.shape[3]
initial_learning_rate = 0.01
validation_split=0.05

model = get_model(iterations=1)
# model.summary()

In [6]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "weights/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=1*batch_size)

decay = initial_learning_rate / epochs
def lr_time_based_decay(epoch, lr):
    return lr * 1 / (1 + decay * epoch)


model_history = model.fit(train_images[0:5000], train_images[0:5000], 
                          batch_size=batch_size, 
                          epochs=epochs, 
                          validation_split=validation_split, 
                          callbacks=[cp_callback])



# # Save the weights using the `checkpoint_path` format
# model.save_weights(checkpoint_path.format(epoch=0))



Epoch 1/5
10/10 [==============================] - 10s 996ms/step - loss: 1.1108 - SSIMLoss: 0.7344 - PSNRLoss: 8.0892 - residual_loss: 0.3407 - mae: 0.3672 - acc: 0.2458 - val_loss: 0.6760 - val_SSIMLoss: 0.5612 - val_PSNRLoss: 11.4459 - val_residual_loss: 0.1148 - val_mae: 0.2352 - val_acc: 0.2331
Epoch 2/5
10/10 [==============================] - 7s 705ms/step - loss: 0.7470 - SSIMLoss: 0.5335 - PSNRLoss: 12.1136 - residual_loss: 0.2032 - mae: 0.2190 - acc: 0.3652 - val_loss: 0.6172 - val_SSIMLoss: 0.5132 - val_PSNRLoss: 12.3559 - val_residual_loss: 0.1040 - val_mae: 0.2130 - val_acc: 0.5028
Epoch 3/5
10/10 [==============================] - 7s 709ms/step - loss: 0.7233 - SSIMLoss: 0.5173 - PSNRLoss: 12.3952 - residual_loss: 0.1968 - mae: 0.2122 - acc: 0.5160 - val_loss: 0.6133 - val_SSIMLoss: 0.5120 - val_PSNRLoss: 12.6361 - val_residual_loss: 0.1013 - val_mae: 0.2075 - val_acc: 0.5028
Epoch 4/5
10/10 [==============================] - 7s 710ms/step - loss: 0.7185 - SSIMLoss: 0.512

In [7]:
path = 'models/new_model'
save_model(model, path)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/new_model/assets
Model saved to: models/new_model.


In [ ]:
custom_objects = {'SSIMLoss': SSIMLoss, 'PSNRLoss': PSNRLoss, 'residual_loss': residual_loss, 'SSIM_res_loss': SSIM_res_loss}
loaded_model = load_model(path, custom_objects)

In [ ]:
test_model(loaded_model, test_images)

In [30]:
model_history